# Integrantes

* Juan Ignacio García (a2008)
* Rodrigo Mesa Marchi (a2016)
* Danilo Simón Reitano Andrades (a2020)

# Desarrollo

## PEAS del Problema

Consideramos que el agente se trata de un agente de software que resolverá el problema de las Torres de Hanoi llevando todos los discos de una posición inicial a una final. Con este contexto en mente se definen los PEAS del problema:

**Performance**: Minimizar cantidad de pasos necesarios hasta encontrar la solución del problema sin incumplir las restricciones dadas.

**Environment**: El rompecabezas de Torres de Hanoi, con 3 varillas, *n* discos; y considerando las siguientes restricciones:
* Solo se puede mover el disco más superficial de cada varilla.
* Solo se puede ubicar un disco más pequeño sobre uno más grande o un espacio vacío.
* Solo se puede hacer un movimiento a la vez.

**Actuators**: Pensando únicamente en un software que solucione este problema, los actuadores son las funciones que permiten mover una pieza entre varillas.

**Sensors**: La clase que representa el nodo actual de la evolución del problema tiene la capacidad de observar el estado en que se encuentran los discos en todo momento.

## Propiedades del Entorno de Trabajo

En el contexto mencionado anteriormente, se definen las siguientes propiedades del entorno:

**Totalmente observable**: Se dispone de los sensores para leer toda la información necesaria del entorno para resolver el problema.

**Determinista**: La selección de un estado sucesor se da en forma determinística, siguiendo un orden dado por la función de evaluación.

**Episódico**: Cada estado es independiente del camino que ha realizado. Dicho de otro modo, se puede encontrar una solución (si existe) comenzando desde cualquier estado posible sin importar cómo se ha llegado al estado inicial. Dicho esto, se pretende incorporar un mecanismo que puede considerarse secuencial que registra los nodos ya visitados para evitar explorarlos nuevamente.

**Estático**: Se considera que no pueden ocurrir cambios indeseados mientras se evalúa o ejecuta el camino encontrado.

**Discreto**: El número de acciones que se pueden tomar para alcanzar un estado sucesor es finito y observable.

**Agente individual**: El problema puede ser resuelto por un único agente sin contar con la intervención de ningún otro.

## Definiciones de Conceptos Importantes

Se incluye la definición de algunos conceptos importantes en la jerga de los algoritmos de búsqueda contextualizados para este problema concreto:

**Estados**: Cada configuración ordenada de los discos contenidos en cada estaca.

**Espacio de estados**: Todas las configuraciones válidas ($3^n - 1$) posibles para el problema de Torres de Hanoi para *n* discos.

**Árbol de Búsqueda**: Es un diagrama de grafos que se forma tomando una configuración inicial del problema, considerando para este nodo todas las posibles acciones que se pueden tomar que modifican el estado actual hacia uno siguiente.

**Nodo de Búsqueda**: Estructura que representa a un estado dentro del espacio de búsqueda junto con información relevante como el nodo padre del que proviene, el costo del camino y el valor de su heurística (si aplica).

**Objetivo**: El estado que describe la configuración de discos a la que se desea llegar.

**Acción**: Mover un disco superficial de una varilla a otra cumpliendo las restricciones. Cuando se aplica una acción se dice que se *expande* el nodo.

**Frontera**: Son los nodos almacenados en la estructura de búsqueda siguiendo un orden dado según el algoritmo de búsqueda elegido. Gráficamente corresponde a las hojas del árbol.

## Implementación del Algoritmo de búsqueda

Se ha notado que todos los algoritmos de búsqueda siguen una estrategia iterativa similar en la que se expande un nodo, se realiza un test objetivo para ver si se alcanzó la meta y, en caso contrario, se introducen los nodos sucesores dentro de la cola de búsqueda. El algoritmo que se utiliza define la prioridad en que se expandirá el siguiente nodo desde la frontera, siendo este mecanismo lo único que diferencia un algoritmo de otro a nivel operativo. Por este motivo se desarrolla una función `global_search_algo` que contiene esta estructura genérica y que recibe como parámetros el número de discos `n` con el que se quiere buscar una solución y un método que calcula el valor de prioridad que se asigna a cada nodo de la frontera, apoyándonos de una estructura de datos del tipo `PriorityQueue` para organizar los nodos.

Se destaca que se prepara el programa para que siempre considere que se partirá de un estado inicial en el que los `n` discos se ubican ordenados en la varilla 1 y se llegue a un estado objetivo donde los discos estén ordenados en la varilla 3. También se prepara un mecanismo que detecte cuando no exista una solución e informe de ello en caso de que ocurra.

In [3]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from queue import PriorityQueue
from aima_libs.tree_hanoi import NodeHanoi

def search_algo(number_disks, evaluation_function):
    # Inicializamos el problema creando una lista de number_disks discos ordenada de mayor a menor
    # Se declara como estado inicial todos los discos en la primera varilla y como estado final todos en la última
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos el nodo raíz del árbol de búsqueda
    initial_node = NodeHanoi(problem.initial)

    # Creamos una cola con prioridad en la que se introduce el nodo inicial. La prioridad del nodo es calculada por la función `evaluation_function` pasada como argumento del método
    frontier = PriorityQueue()
    frontier.put((evaluation_function(initial_node), initial_node))

    # Creamos el set con estados ya visitados
    explored = set()
    nodes_explored = 0
    max_depth = 0
    
    # Iteramos en el espacio de búsqueda hasta que se agoten los nodos en la cola de búsqueda
    while not frontier.empty():
        # Adoptamos como nodo actual al primero en la cola
        node = frontier.get()[1]
        nodes_explored += 1
        if node.depth > max_depth:
            max_depth = node.depth 
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": nodes_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": frontier.qsize(),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual. El método `expand` verifica internamente que los estados que se añaden a la cola son válidos
        for next_node in node.expand(problem):
            # Si el estado sucesor que se evalúa ya ha sido visitado, se ignora
            if next_node.state not in explored:
                # La prioridad del nodo es calculada por la función `evaluation_function` pasada como argumento del método
                frontier.put((evaluation_function(next_node), next_node))

    # Si no se encontró la solución, devolvemos las métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": nodes_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": frontier.qsize(),
        "max_depth": max_depth,
        "cost_total": None,
    }
    return None, metrics

Habiendo definido la estructura común que siguen los algoritmos, se definen las funciones para ejecutar tres estrategias de búsqueda distintas: La estrategia A*, la avara y la de costo uniforme (también llamada de Dijkstra). 

Como los dos primeros algoritmos son de búsqueda informada, resulta necesario diseñar una función *heurística*, que indique si un nodo es favorable frente a otro para alcanzar la solución. La heurística diseñada toma la cantidad de discos ubicados en la tercera estaca, ponderado por el tamaño del disco que se encuentre en la estaca. Esto es:

$h(x) = \sum_{i=1}^{n} i, \quad \forall i \in \text{peg}_3$

La complejidad espacial y temporal de cada algoritmo elegido, así como si es completo y óptimo, se sintetiza en la siguiente tabla:

| Algoritmo       | Complejidad en tiempo        | Complejidad en memoria | Completo | Óptimo |
|-----------------|-----------------------------|------------------------|------------------------|------------------------|
| **A***         | $O(b^d)$ (peor caso)       | $O(b^d)$             | Sí   | Sí    |
| **Greedy** | $O(b^d)$ (peor caso) | $O(b^d)$             | Sí | No    |
| **Dijkstra**   | $O(\|A\|\log(V))$ (usando una cola prioritaria)| $O(\|V\|)$ | Sí | Sí    |

Siendo:

* *b* el factor de ramificación (número de hijos por nodo en promedio).

* *d* la profundidad del nodo objetivo.

* ∣*V*∣ el número de vértices en el grafo.

* ∣*A*∣ el número de aristas en el grafo.

Los métodos que definen la función de evaluación de cada algoritmo se definen a continuación:

In [4]:
def a_star_evaluation(node: NodeHanoi) -> int:
    """
    La función de evaluación del algoritmo A* se compone de la suma del costo del camino y el valor de la función heurística
    """
    return node.path_cost - sum(node.state.get_state()[2])

def greedy_evaluation(node: NodeHanoi) -> int:
    """
    La función de evaluación del algoritmo avaro se compone del valor de la función heurística
    """
    return -sum(node.state.get_state()[2])

def dijkstra_evaluation(node: NodeHanoi) -> int:
    """
    La función de evaluación del algoritmo de costo uniforme se compone del costo del camino
    """
    return node.path_cost

Una vez definida cada función de evaluación, se testea el problema con `n=7` discos para cada estrategia, corriendo el programa 10 veces y midiendo el tiempo promedio de ejecución y su distribución estándar:

In [5]:
%%timeit -r 10 -n 1
problem_a_star, metrics_a_star = search_algo(7, a_star_evaluation)

89.5 ms ± 3.57 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [6]:
%%timeit -r 10 -n 1
problem_greedy, metrics_greedy = search_algo(7, greedy_evaluation)

86.5 ms ± 853 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [7]:
%%timeit -r 10 -n 1
problem_dij, metrics_dij = search_algo(7, dijkstra_evaluation)

140 ms ± 1.05 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


Para medir la memoria utilizada por cada algoritmo se usa la librería `tracemalloc`, que permite obtener una lectura del pico de memoria consumida en la ejecución de una celda de código:

In [8]:
import tracemalloc
import numpy as np

def memory_measure(evaluation_function):
    measures = []
    for _ in range(10):
        tracemalloc.start()
        
        _, _ = search_algo(7, evaluation_function)
        
        _, memory_peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Convert to MB
        memory_peak /= 1024 * 1024
        measures.append(memory_peak)

    # Calcular estadísticas
    memory_mean = np.mean(measures)
    memory_std = np.std(measures)

    print(f"Pico de memoria promedio: {memory_mean:.2f} MB")
    print(f"Desviación estándar: {memory_std:.2f} MB")

print("A*")
memory_measure(a_star_evaluation)

print("Greedy")
memory_measure(greedy_evaluation)

print("Dijkstra")
memory_measure(dijkstra_evaluation)

A*
Pico de memoria promedio: 1.41 MB
Desviación estándar: 0.04 MB
Greedy
Pico de memoria promedio: 1.59 MB
Desviación estándar: 0.03 MB
Dijkstra
Pico de memoria promedio: 2.31 MB
Desviación estándar: 0.05 MB


Las métricas para cada algoritmo son las siguientes:

In [9]:
_, metrics_a_star = search_algo(7, a_star_evaluation)
print(metrics_a_star)
print(f"Diferencia con la solución óptima: {metrics_a_star['cost_total']-(2**7-1)}")

{'solution_found': True, 'nodes_explored': 2509, 'states_visited': 1494, 'nodes_in_frontier': 48, 'max_depth': 127, 'cost_total': 127.0}
Diferencia con la solución óptima: 0.0


In [10]:
_, metrics_greedy = search_algo(7, greedy_evaluation)
print(metrics_greedy)
print(f"Diferencia con la solución óptima: {metrics_greedy['cost_total']-(2**7-1)}")

{'solution_found': True, 'nodes_explored': 2547, 'states_visited': 1449, 'nodes_in_frontier': 95, 'max_depth': 138, 'cost_total': 138.0}
Diferencia con la solución óptima: 11.0


In [11]:
_, metrics_dij = search_algo(7, dijkstra_evaluation)
print(metrics_dij)
print(f"Diferencia con la solución óptima: {metrics_dij['cost_total']-(2**7-1)}")

{'solution_found': True, 'nodes_explored': 4071, 'states_visited': 2072, 'nodes_in_frontier': 208, 'max_depth': 127, 'cost_total': 127.0}
Diferencia con la solución óptima: 0.0


La síntesis de los resultados obtenidos se pueda observar en la siguiente tabla:

| Algoritmo         | Tiempo Promedio        | Desviación de Tiempos | Memoria Promedio | Desviación de Memorias | Lejanía a Solución Optima |
|-------------------|------------------------|-----------------------|------------------|------------------------|---------------------------|
| **A***            | 85.9 ms                | 893 μs                | 1.41 MB          | 0.04 MB                | 0                         |
| **Greedy**        | 87.3 ms                | 667 μs                | 1.59 MB          | 0.03 MB                | 11                        |
| **Dijkstra**      | 140 ms                 | 1.13 ms               | 2.31 MB          | 0.04 MB                | 0                         |

En conclusión se observa que el algoritmo de Dijkstra es más sencillo de implementar ya que no requiere de idear una función heurística; sin embargo explora casi el doble de estados que los otros algoritmos, lo que queda en evidencia por los mayores tiempos de ejecución y mayor cantidad de memoria empleada observados.

Los algoritmos A* y Greedy performan de manera bastante similar, siendo el A* ligeramente más rápido y liviano, pero se observa que el algoritmo Greedy no siempre encuentra la mejor solución.

## Simulación

Se incluye una forma de simular el problema utilizando la herramienta de visualización proporcionada por la cátedra. Para utilizarlo es necesario cambiar el parámetro de la función de evaluación que se desea aplicar (aunque según los experimentos realizados, solo debería verse una solución distinta en el algoritmo Greedy); esto generará los archivos JSON necesarios ppara que el simulador reconozca cuál es el estado inicial y el plan de acciones que debe seguir. Tras ejecutar esta celda, debe ejecutarse manualmente el archivo `simulator/simulation_hanoi.py` para visualizar el problema.

In [15]:
import shutil
import os

evaluation_function = greedy_evaluation

solution, _ = search_algo(7, evaluation_function)
solution.generate_solution_for_simulator()

# Definir archivos de simulación y directorio de destino
archivo1 = "initial_state.json"
archivo2 = "sequence.json"
destino = "simulator"

# Mover archivos
shutil.move(archivo1, os.path.join(destino, archivo1))
shutil.move(archivo2, os.path.join(destino, archivo2))

'simulator\\sequence.json'